In [ ]:
import pandas as pd
import dask.dataframe as dd
df_pacientes = pd.read_csv('hosp/patients.csv')
df_admissoes = pd.read_csv('hosp/admissions.csv')
df_diagnosticos = pd.read_csv('hosp/diagnoses_icd.csv')
df_d_diagnosticos = pd.read_csv("hosp/d_icd_diagnoses.csv")

df_estadia_ed = pd.read_csv('D:\MIMIC\mimic-iv-ed-2.2\ed\edstays.csv')[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']]
df_diagnosticos_ed = pd.read_csv('D:\MIMIC\mimic-iv-ed-2.2\ed\diagnosis.csv')[['subject_id', 'stay_id', 'icd_code', 'icd_title']]
df_sinais_vitais_ed = pd.read_csv('D:/MIMIC/mimic-iv-ed-2.2/ed/vitalsign.csv')[['subject_id', 'stay_id', 'charttime','temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'rhythm']]

df_bio = pd.read_csv('hosp\microbiologyevents.csv')

# pacientes_qSOFA_2 = pd.read_excel('pacientes_qSOFA_2.xlsx', index_col=0)
# pacientes_diag_seps = pd.read_excel('pacientes_diag_sepse.xlsx', index_col=0)
# pacientes = pacientes_qSOFA_2.merge(pacientes_diag_seps, on=['subject_id', 'hadm_id'])
pacientes = pd.read_excel('pacientes_sepse_qSOFA_2.xlsx', index_col=0)

## Função

In [ ]:
df_final = df_admissoes.merge(df_pacientes, how='left', on='subject_id')
df_final = df_final[['subject_id', 'hadm_id','gender', 'anchor_age', 'dod', 'admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']]
df_final =  df_final.merge(df_estadia_ed, how='left', on=['subject_id', 'hadm_id'])
#df_final = df_final[(df_final['subject_id']==subject_id)&(df_final['hadm_id']==hadm_id)][['subject_id','hadm_id','stay_id','admittime','dischtime','deathtime','edregtime','edouttime','intime','outtime']]
#df_final[(df_final['subject_id']==subject_id)][['subject_id','hadm_id','stay_id','admittime','dischtime','deathtime','edregtime','edouttime','intime','outtime']]
df_final = df_final[['subject_id','hadm_id','stay_id','admittime','dischtime','deathtime','edregtime','edouttime','intime','outtime']]
# Filtrar somente pacientes qSOFA>=2
df_final = df_final.merge(pacientes[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'])

In [ ]:
def reformat_table(df):
    log_data = []
    for _, row in df.iterrows():
        log_data.append([row["subject_id"], row["hadm_id"], row["stay_id"], "Register the time of in-hospital death for the patient", "deathtime", "admissions/hosp", row["deathtime"]])
        log_data.append([row["subject_id"], row["hadm_id"], row["stay_id"], "Register the date and time the patient was admitted to the hospital", "admittime", "admissions/hosp", row["admittime"]])
        log_data.append([row["subject_id"], row["hadm_id"], row["stay_id"], "Register the date and time the patient was discharged from the hospital", "dischtime", "admissions/hosp", row["dischtime"]])
        log_data.append([row["subject_id"], row["hadm_id"], row["stay_id"], "Register the admission datetime of the given emergency department stay", "intime", "edstays/ed", row["intime"]])
        log_data.append([row["subject_id"], row["hadm_id"], row["stay_id"], "Register the discharge datetime of the given emergency department stay", "outtime", "edstays/ed", row["outtime"]])
        log_data.append([row["subject_id"], row["hadm_id"], row["stay_id"], "Register the date and time at which the patient was registered in the emergency department", "edregtime", "admissions/hosp", row["edregtime"]])
        log_data.append([row["subject_id"], row["hadm_id"], row["stay_id"], "Register The date and time at which the patient was discharged from the emergency department", "edouttime", "admissions/hosp", row["edouttime"]])

    new_df = pd.DataFrame(log_data, columns=["subject_id", "hadm_id", "stay_id", "Activity", "Coluna", "Resource", "Timestamp"])
    
    return new_df

In [ ]:
df_transformed = reformat_table(df_final)
df_transformed

In [ ]:
df_final = df_final.merge(df_sinais_vitais_ed, how='left', on=['subject_id', 'stay_id'])
df = df_final[['subject_id','hadm_id','stay_id','charttime','resprate','sbp']]

In [ ]:
def transform_row(row):
    return {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'stay_id': row['stay_id'],
        'Activity': "Measure the patient's vital signs.",
        'Coluna': 'charttime',
        'Resource': 'vitalsign/ed',
        'Timestamp': row['charttime'],
        'Info adc': f" Resprate {row['resprate']} - SBP {row['sbp']}"
    }

df_transformed2 = df_final.apply(transform_row, axis=1)
df_transformed2 = pd.DataFrame(df_transformed2.tolist())
df_transformed2

In [ ]:
df_remedios_pyxis = pd.read_csv('ed\pyxis.csv')
#df_remedios_ed = df_remedios_pyxis
df_remedios_pyxis = df_remedios_pyxis.merge((pd.DataFrame((pacientes['subject_id'].unique()),columns=['subject_id'])), on=['subject_id'])

In [ ]:
df_remedios_de_para_pyxis = pd.read_excel('remedios_de_para_pyxis.xlsx', index_col=0)
df_remedios_de_para_pyxis['Tipo'] = df_remedios_de_para_pyxis.apply(lambda row: 'Antibiotic' if row['Tipo'] == 'Antibiótico' else row['Tipo'], axis = 1)
df_remedios_pyxis = df_remedios_pyxis.merge(df_remedios_de_para_pyxis, on=['name'])

In [ ]:
columns_transformed = ['subject_id', 'stay_id', 'Activity', 'Coluna', 'Timestamp','Info adc']
df_transformed3 = pd.DataFrame(columns=columns_transformed)

def transform_row(row):
    return {
        'subject_id': row['subject_id'],
        'stay_id': row['stay_id'],
        'Activity': f'Administer medication {row["Tipo"]} via Pyxis in the emergency',
        'Coluna': 'charttime',
        'Resource': 'pyxis/ed',
        'Timestamp': row['charttime'],
        'Info adc': row['name']
    }

df_transformed3 = df_remedios_pyxis[['subject_id','stay_id','charttime','name','Tipo']].apply(transform_row, axis=1)
df_transformed3 = pd.DataFrame(df_transformed3.tolist())

In [ ]:
df_transformed3 = df_transformed3[['subject_id','stay_id','Activity','Coluna','Resource','Timestamp','Info adc']]
df_transformed3

In [ ]:
df_transformed_join = df_transformed[['subject_id','hadm_id','stay_id']].astype({'stay_id': 'int32'}).astype({'stay_id': 'object'})
df_transformed_join = df_transformed_join[['subject_id','hadm_id','stay_id']].drop_duplicates()
df_transformed3 = df_transformed3.merge(df_transformed_join[['subject_id','hadm_id','stay_id']], on=['subject_id','stay_id'], how='inner')[['subject_id','hadm_id', 'stay_id','Activity','Coluna','Resource','Timestamp','Info adc']]

In [ ]:
df_d_items = pd.read_csv('icu\d_items.csv')

In [ ]:
# TABELA LABEVENTS
ddf = dd.read_csv('particoes_labevents_filtradas_sep_qSOFA2\particao_*.csv', dtype='object')
pandas_labeventos = ddf.compute()
pandas_labeventos = pandas_labeventos.astype({'itemid': 'int64'})

In [ ]:
df_dm_labevents = pd.read_csv('hosp/d_labitems.csv')
pandas_labeventos = pandas_labeventos.merge(df_dm_labevents, on=['itemid'])

In [ ]:
pandas_labeventos = pandas_labeventos[pandas_labeventos['label']=='Lactate']

In [ ]:
pandas_labeventos

In [ ]:
def transform_row(row):
    return {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'stay_id': '',
        'Activity': f"Record laboratory measurement/observation {row['label']}",
        'Coluna': 'charttime',
        'Resource': 'labevents/hosp',
        'Timestamp': row['charttime'],
        'Info adc': f"{row['label']} - {row['value']} {row['valueuom']}"
    }

df_transformed4 = pandas_labeventos.apply(transform_row, axis=1)
df_transformed4 = pd.DataFrame(df_transformed4.tolist())
df_transformed4

In [ ]:
df_uti = pd.read_csv('icu\icustays.csv')
df_uti = df_uti.merge(pacientes[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'])
df_uti = df_uti.rename(columns={'stay_id': "stay_id_icu"})
df_uti

In [ ]:
columns_transformed = ['subject_id', 'hadm_id','stay_id_icu', 'Activity', 'Coluna','Resource', 'Timestamp']
df_transformed5 = pd.DataFrame(columns=columns_transformed)

def transform_row(row):
    intime_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'stay_id_icu': row['stay_id_icu'],
        'Activity': 'Record the date and time the patient was transferred to the ICU.',
        'Coluna': 'intime',
        'Resource': 'icustays/icu',
        'Timestamp': row['intime']
    }
    outtime_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'stay_id_icu': row['stay_id_icu'],
        'Activity': 'Record the date and time the patient was transferred from the ICU.',
        'Coluna': 'outtime',
        'Resource': 'icustays/icu',
        'Timestamp': row['outtime']
    }

    return [intime_row, outtime_row]

rows_transformed = df_uti.apply(transform_row, axis=1)
df_transformed5 = pd.DataFrame([row for row_pair in rows_transformed for row in row_pair])

In [ ]:
df_transformed5

In [ ]:
df_presc = pd.read_csv("hosp\prescriptions.csv")
df_presc = df_presc.merge(pacientes[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'])
df_presc = df_presc[['subject_id','hadm_id','starttime','stoptime','drug','dose_val_rx','dose_unit_rx']]

In [ ]:
df_remedios_de_para = pd.read_excel('remedios_de_para.xlsx', index_col=0)
df_remedios_de_para['Tipo'] = df_remedios_de_para.apply(lambda row: 'Antibiotic' if row['Tipo'] == 'Antibiótico' else row['Tipo'], axis = 1)

In [ ]:
df_presc = df_presc.merge(df_remedios_de_para, on=['drug'])
df_presc['Tipo'] = df_presc.apply(lambda row: 'Albumin' if row['Albumina'] == 'Sim' else row['Tipo'], axis = 1)

In [ ]:
def transform_row(row):
    starttime_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'stay_id': '',
        'Activity': f"Record the prescribed time for the medication {row['Tipo']}",
        'Coluna': 'starttime/stoptime',
        'Resource': 'prescriptions/hosp',
        'Init Timestamp': row['starttime'],
        'End Timestamp': row['stoptime'],
        'Info adc': f"{row['drug']} - {row['dose_val_rx']} {row['dose_unit_rx']}"
    }
    return [starttime_row]

rows_transformed = df_presc.apply(transform_row, axis=1)
df_transformed6 = pd.DataFrame([row for row_pair in rows_transformed for row in row_pair])

In [ ]:
df_transformed6

In [ ]:
procedimentos_imp = [
    'Invasive Ventilation',
    'Blood Cultured',
    'Urine Culture',
    'Sputum Culture',
    'Dialysis Catheter',
    'Stool Culture',
    'Pan Culture',
    'BAL Fluid Culture',
    'Peritoneal Dialysis',
    'Wound Culture',
    'CSF Culture'
]
df_proced_imp = pd.DataFrame(procedimentos_imp, columns=['label'])

In [ ]:
df_procedimentos = pd.read_csv('icu\procedureevents.csv')
df_procedimentos = df_procedimentos.merge(df_d_items, how = 'left', on='itemid')
df_procedimentos = df_procedimentos.merge(pacientes[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'])
df_procedimentos = df_procedimentos[['subject_id','hadm_id','stay_id','starttime','endtime', 'value', 'valueuom', 'label', 'abbreviation', 'category']].rename(columns={'stay_id': 'stay_id_icu'})
df_procedimentos = df_procedimentos.merge(df_proced_imp, on='label')

In [ ]:
def transform_row(row):
    starttime_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'stay_id_icu': row['stay_id_icu'],
        'Activity': f"Record the procedure {row['label']}.",
        'Coluna': 'starttime/endtime',
        'Resource': 'procedureevents/icu',
        'Init Timestamp': row['starttime'],
        'End Timestamp': row['endtime'],
        'Info adc': f"{row['category']} - {row['value']} {row['valueuom']}"
    }
    return [starttime_row]

rows_transformed = df_procedimentos.apply(transform_row, axis=1)
df_transformed7 = pd.DataFrame([row for row_pair in rows_transformed for row in row_pair])

In [ ]:
df_transformed7

In [ ]:
# MICROBIOLOGIA, HEMOCULTURA
df_bio = df_bio[df_bio['spec_type_desc']=='BLOOD CULTURE']
df_bio = df_bio.merge(pacientes, on=['subject_id', 'hadm_id'])

In [ ]:
df_bio

In [ ]:
def transform_row(row):
    starttime_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'Activity': f"Record the date and time of the routine Blood Culture",
        'Coluna': 'charttime',
        'Resource': 'microbiologyevents/hosp',
        'Init Timestamp': row['charttime'],
        'End Timestamp': row['charttime'],
        'Info adc': ''
    }
    return [starttime_row]

rows_transformed = df_bio.apply(transform_row, axis=1)
df_transformed8 = pd.DataFrame([row for row_pair in rows_transformed for row in row_pair])

In [ ]:
df_transformed8

In [ ]:
#df_paciente = pd.concat([df_transformed, df_transformed2, df_transformed3, df_transformed4, df_transformed5, df_transformed6, df_transformed7],ignore_index=True)
df_paciente = pd.concat([df_transformed, df_transformed2, df_transformed3, df_transformed4, df_transformed5, df_transformed6, df_transformed7, df_transformed8],ignore_index=True)
df_paciente

In [ ]:
df_paciente['subject_id'] = df_paciente['subject_id'].apply(lambda x: int(x))
df_paciente['hadm_id'] = df_paciente['hadm_id'].apply(lambda x: int(float(x)))

In [ ]:
# df_paciente = df_paciente.rename(columns={"Tipo":"Activity","Info adc":"Add. Info","Coluna":"Column"})
df_paciente = df_paciente.rename(columns={"Info adc":"Add. Info","Coluna":"Column"})
df_paciente = df_paciente[['subject_id','hadm_id','stay_id','stay_id_icu','Activity','Column','Resource','Timestamp','Init Timestamp','End Timestamp','Add. Info']]

In [ ]:
#df_paciente = df_paciente[df_paciente['Timestamp'].notna()]
df_paciente['Init Timestamp'] = df_paciente.apply(lambda row: row['Init Timestamp'] if type(row['Timestamp'])==float else row['Timestamp'], axis=1)
df_paciente['End Timestamp'] = df_paciente.apply(lambda row: row['End Timestamp'] if type(row['Timestamp'])==float else row['Timestamp'], axis=1)
df_paciente = df_paciente[df_paciente['Init Timestamp'].notna()]

In [ ]:
df_paciente

In [ ]:
# Adicionar diagnóstico
df_pacientes_sepse = pd.read_excel('pacientes_diag_sepse.xlsx', index_col=0)
df_B_filtered = df_paciente[df_paciente['Activity'] == 'Register the admission datetime of the given emergency department stay']
df_merged = pd.merge(df_pacientes_sepse, df_B_filtered, how='inner', on=['subject_id', 'hadm_id'])
df_merged = df_merged[['subject_id','hadm_id','Activity','Column','Resource','Init Timestamp','End Timestamp','long_title']]
df_merged.rename(columns={'long_title':"Add. Info"}, inplace=True)

df_merged['Activity'] = 'Register the datetime of the Sepsis diagnosis'
df_merged['Column'] = 'long_title'
df_merged['Resource'] = 'hosp/d_icd_diagnoses'

df_merged2 = pd.merge(df_paciente, df_merged, how='left', on=['subject_id', 'hadm_id'])

df_paciente = pd.concat([df_paciente, df_merged],ignore_index=True)

In [ ]:
df_paciente

In [ ]:
len(df_paciente['subject_id'].unique())

In [ ]:
# Excluir a atividade de alta se o paciente morreu

# Encontre os registros com deathtime
deathtime_rows = df_paciente[df_paciente['Column'] == 'deathtime']

# Crie um conjunto de tuplas (subject_id, hadm_id) para pacientes que morreram
deceased_patients = set(zip(deathtime_rows['subject_id'], deathtime_rows['hadm_id']))

# Função para verificar se uma linha deve ser mantida
def keep_row(row):
    patient_info = (row['subject_id'], row['hadm_id'])
    return not (patient_info in deceased_patients and row['Column'] == 'dischtime')

# Filtra as linhas usando a função keep_row
filtered_df = df_paciente[df_paciente.apply(keep_row, axis=1)]

In [ ]:
filtered_df

In [ ]:
filtered_df.drop(columns=['Timestamp'],inplace=True)

In [ ]:
filtered_df

In [ ]:
filtered_df['Init Timestamp'] = pd.to_datetime(filtered_df['Init Timestamp'])
filtered_df['End Timestamp'] = pd.to_datetime(filtered_df['End Timestamp'])

In [ ]:
# Identificar as atividades de alta ou morte
important_activities = ["Register the date and time the patient was discharged from the hospital",
                        "Register the time of in-hospital death for the patient"]

# Criar um dataframe apenas com as atividades de alta ou morte
df_important = filtered_df[filtered_df['Activity'].isin(important_activities)]

# Encontrar o momento da primeira atividade de alta ou morte para cada paciente e internação
important_times = df_important.groupby(['subject_id', 'hadm_id'])['Init Timestamp'].min()

# Converter a Series em um DataFrame para facilitar o merge
important_times = important_times.reset_index()

# Mudar o nome da coluna para evitar conflito no merge
important_times.rename(columns={'Init Timestamp': 'Important Time'}, inplace=True)

# Fazer o merge do df original com os tempos importantes
df = pd.merge(filtered_df, important_times, on=['subject_id', 'hadm_id'], how='left')

# Remover as linhas onde o tempo inicial do evento é depois do tempo importante
df = df[df['Init Timestamp'] <= df['Important Time']]

# Remover a coluna auxiliar
df.drop(columns='Important Time', inplace=True)

In [ ]:
df

In [ ]:
# Retirar a atividade 'Register the date and time the patient was admitted to the hospital'
df = df[df['Activity']!='Register the date and time the patient was admitted to the hospital']

In [ ]:
# Mudar nome do arquivo
# df_paciente.sort_values(by='Timestamp').to_excel('pacientes_finais_200.xlsx')
# filtered_df.sort_values(by=['subject_id','hadm_id','Timestamp']).reset_index(drop=True).to_excel('pacientes_finais_17-05.xlsx')
df.sort_values(by=['subject_id','hadm_id','Init Timestamp']).reset_index(drop=True).to_csv('pacientes_finais_DATA.csv', sep=';')